## CNN sentence classifier - Round 2 of Hyperparameter tuning with balanced data

Following the analysis of round 1, this is a fine tuning of the hyperparameters.

Note that data load, model creation and training have been broken out into jbyrne_utils.py to simplify the notebook.

In [1]:
## Enable Tensorboard
%load_ext tensorboard
## force autoreload of changed modules
%load_ext autoreload
%autoreload 2

## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 100


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


In [5]:
for dict in d:
    if dict["label"] == 1:
        print(dict["text"])
        print()

And when you look at your tax policies that are directed primarily at those who are doing well, and you are neglecting people who are really struggling right now, I think that is a continuation of the last eight years, and we can't afford another four.

Mali has been taken over, the northern part of Mali by al-Qaeda type individuals.

And so I've put forward a specific $4 trillion deficit reduction plan.

In '17, it implodes by itself.

Now, if you believe The Washington Post and you believe this extensive study that's been done -- and I'm reading it -- right after election day this year they're going to hit us with a hundred banks, it will be a $100-billion problem.

Five hundred thousand kids lost after-school programs because of your budget.

See, we've got a $4 trillion debt and only in America would you have $2.8 trillion of it or 70% of it financed 5 years or less.

We're not allowed to vote on it.

And you can take it out on charts, you look at all the studies the different grou

## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [6]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 9.06 ms, sys: 12.5 ms, total: 21.6 ms
Wall time: 21.1 ms


## Tokenize the training data set and build a vocabulary from it
Want to do this to minimize the vocab to just what is in the training set for performance.

using defaults:  strip all punctuation except apostrophies, set to lower case and split on spaces. No restriction on length of vocabulary.

In [7]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, tokenizer = tokenize_sentences(x_train, max_len)
x_val_ids, tokenizer   = tokenize_sentences(x_val, max_len)


Loading previously created Tokenizer
Loading previously created Tokenizer


In [8]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [9]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim)

Loading previously created Embedding Matrix
CPU times: user 1.68 ms, sys: 391 µs, total: 2.07 ms
Wall time: 1.7 ms


In [10]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


## Output of the tests are deleted as the notebook will not load in Chrome

In [11]:

# this is very quick, so we can use grid search on these models.

run_logfile = "runs2bal.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models
for k_sizes in [ [8,16], [16,32], [32,64], [4,8,12], [4,8,16], [8,12,16], [8,16,32], [4,8,16,32], [4,8,32,64] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96, 96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal",
                      model_dir        = "../models/CNNBal")
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 4s 8ms/step - loss: 0.6560 - accuracy: 0.6060 - val_loss: 0.5802 - val_accuracy: 0.7043
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-202739/assets
Epoch 2/20
288/288 [==============================] - 2s 5ms/step - loss: 0.4533 - accuracy: 0.7880 - val_loss: 0.4193 - val_accuracy: 0.8169
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-202739/assets
Epoch 3/20
288/288 [==============================] - 2s 5ms/step - loss: 0.2909 - accuracy: 0.8806 - val_loss: 0.3433 - val_accuracy: 0.8689
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-202739/assets
Epoch 4/20
288/288 [==============================] - 2s 5ms/step - loss: 0.1894 - accuracy: 0.9307 - val_loss: 0.2623 - val_accuracy: 0.9055
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-202739/assets
Epoch 5/20
288/288 [=============

INFO:tensorflow:Assets written to: ../models/CNNBal/210409-202854/assets
Epoch 5/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0915 - accuracy: 0.9672 - val_loss: 0.2863 - val_accuracy: 0.9078
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-202854/assets
Epoch 6/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0600 - accuracy: 0.9802 - val_loss: 0.2192 - val_accuracy: 0.9331
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-202854/assets
Epoch 7/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0501 - accuracy: 0.9857 - val_loss: 0.4181 - val_accuracy: 0.8969
Epoch 8/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0391 - accuracy: 0.9860 - val_loss: 0.3304 - val_accuracy: 0.9195
Epoch 9/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0282 - accuracy: 0.9915 - val_loss: 0.2912 - val_accuracy: 0.9281
Epoch 10/20
288/288 [==============================] - 2s 5ms/step - lo

288/288 [==============================] - 2s 5ms/step - loss: 0.0087 - accuracy: 0.9968 - val_loss: 0.4933 - val_accuracy: 0.9236
Epoch 19/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0147 - accuracy: 0.9950 - val_loss: 0.4603 - val_accuracy: 0.9146
Epoch 20/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0074 - accuracy: 0.9982 - val_loss: 0.3399 - val_accuracy: 0.9453



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 7ms/step - loss: 0.6354 - accuracy: 0.6438 - val_loss: 0.4571 - val_accuracy: 0.7866
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-203154/assets
Epoch 2/20
288/288 [==============================] - 2s 5ms/step - loss: 0.4065 - accuracy: 0.8215 - val_loss: 0.3493 - val_accuracy: 0.8531
INFO:tensorflow:Assets written to: ../

INFO:tensorflow:Assets written to: ../models/CNNBal/210409-203306/assets
Epoch 4/20
288/288 [==============================] - 2s 5ms/step - loss: 0.1644 - accuracy: 0.9399 - val_loss: 0.2208 - val_accuracy: 0.9231
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-203306/assets
Epoch 5/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0939 - accuracy: 0.9684 - val_loss: 0.1823 - val_accuracy: 0.9417
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-203306/assets
Epoch 6/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0595 - accuracy: 0.9833 - val_loss: 0.2392 - val_accuracy: 0.9313
Epoch 7/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0374 - accuracy: 0.9881 - val_loss: 0.2234 - val_accuracy: 0.9376
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0337 - accuracy: 0.9899 - val_loss: 0.3524 - val_accuracy: 0.9191
Epoch 9/20
288/288 [==============================] - 2s 5ms/step - los

288/288 [==============================] - 2s 5ms/step - loss: 0.0150 - accuracy: 0.9959 - val_loss: 0.2104 - val_accuracy: 0.9571
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-203531/assets
Epoch 17/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0146 - accuracy: 0.9959 - val_loss: 0.3222 - val_accuracy: 0.9412
Epoch 18/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0136 - accuracy: 0.9952 - val_loss: 0.3782 - val_accuracy: 0.9354
Epoch 19/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0151 - accuracy: 0.9957 - val_loss: 0.4541 - val_accuracy: 0.9290
Epoch 20/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0089 - accuracy: 0.9969 - val_loss: 0.4368 - val_accuracy: 0.9299



##########################################################################



Adding Convolution: Kernel Size: 16, Filter Count: 16
Adding Convolution: Kernel Size: 32, Filter Count: 16
Epoch 1/20
288/288 [===============

...

Epoch 11/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0258 - accuracy: 0.9922 - val_loss: 0.2259 - val_accuracy: 0.9444
Epoch 12/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0266 - accuracy: 0.9909 - val_loss: 0.2690 - val_accuracy: 0.9372
Epoch 13/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0240 - accuracy: 0.9918 - val_loss: 0.2961 - val_accuracy: 0.9390
Epoch 14/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0154 - accuracy: 0.9958 - val_loss: 0.3245 - val_accuracy: 0.9286
Epoch 15/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0197 - accuracy: 0.9942 - val_loss: 0.4024 - val_accuracy: 0.9109
Epoch 16/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0215 - accuracy: 0.9930 - val_loss: 0.3308 - val_accuracy: 0.9335
Epoch 17/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0180 - accuracy: 0.9952 - val_loss: 0.2686 - val_accuracy: 0.9507

288/288 [==============================] - 3s 8ms/step - loss: 0.6486 - accuracy: 0.6305 - val_loss: 0.5123 - val_accuracy: 0.7455
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220046/assets
Epoch 2/20
288/288 [==============================] - 2s 7ms/step - loss: 0.4011 - accuracy: 0.8205 - val_loss: 0.4030 - val_accuracy: 0.8232
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220046/assets
Epoch 3/20
288/288 [==============================] - 2s 7ms/step - loss: 0.2161 - accuracy: 0.9187 - val_loss: 0.3146 - val_accuracy: 0.8707
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220046/assets
Epoch 4/20
288/288 [==============================] - 2s 7ms/step - loss: 0.1155 - accuracy: 0.9597 - val_loss: 0.1683 - val_accuracy: 0.9358
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220046/assets
Epoch 5/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0583 - accuracy: 0.9818 - val_loss: 0.2174 - val_accuracy: 0.9259
Epoch 6/2

Epoch 7/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0116 - accuracy: 0.9983 - val_loss: 0.3239 - val_accuracy: 0.9182
Epoch 8/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0059 - accuracy: 0.9995 - val_loss: 0.1986 - val_accuracy: 0.9494
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220215/assets
Epoch 9/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0041 - accuracy: 0.9998 - val_loss: 0.2827 - val_accuracy: 0.9349
Epoch 10/20
288/288 [==============================] - 2s 8ms/step - loss: 0.0071 - accuracy: 0.9985 - val_loss: 0.3036 - val_accuracy: 0.9354
Epoch 11/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0054 - accuracy: 0.9990 - val_loss: 0.2914 - val_accuracy: 0.9394
Epoch 12/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0059 - accuracy: 0.9984 - val_loss: 0.4182 - val_accuracy: 0.9272
Epoch 13/20
288/288 [==============================] - 2s 8ms/step - los

288/288 [==============================] - 3s 8ms/step - loss: 0.6758 - accuracy: 0.5904 - val_loss: 0.6353 - val_accuracy: 0.6609
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220614/assets
Epoch 2/20
288/288 [==============================] - 2s 7ms/step - loss: 0.4465 - accuracy: 0.7908 - val_loss: 0.3322 - val_accuracy: 0.8558
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220614/assets
Epoch 3/20
288/288 [==============================] - 2s 7ms/step - loss: 0.2398 - accuracy: 0.9055 - val_loss: 0.2302 - val_accuracy: 0.9091
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220614/assets
Epoch 4/20
288/288 [==============================] - 2s 7ms/step - loss: 0.1350 - accuracy: 0.9490 - val_loss: 0.2227 - val_accuracy: 0.9127
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220614/assets
Epoch 5/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0833 - accuracy: 0.9746 - val_loss: 0.2535 - val_accuracy: 0.9191
INFO:tens

Epoch 5/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0758 - accuracy: 0.9751 - val_loss: 0.1803 - val_accuracy: 0.9390
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220747/assets
Epoch 6/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0472 - accuracy: 0.9846 - val_loss: 0.1145 - val_accuracy: 0.9602
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220747/assets
Epoch 7/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0367 - accuracy: 0.9880 - val_loss: 0.2403 - val_accuracy: 0.9372
Epoch 8/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0324 - accuracy: 0.9896 - val_loss: 0.3029 - val_accuracy: 0.9268
Epoch 9/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0177 - accuracy: 0.9946 - val_loss: 0.1158 - val_accuracy: 0.9684
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-220747/assets
Epoch 10/20
288/288 [==============================] - 2s 7ms/step - lo





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/